In [1]:
import pandas as pd
import numpy as np
import math

In [5]:
data = pd.read_csv('GroceryStore.csv')

In [6]:
data.shape

(20, 1)

In [7]:
data

,Item
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"
5,"BREAD,TEA,BOURNVITA"
6,"MAGGI,TEA,CORNFLAKES"
7,"MAGGI,BREAD,TEA,BISCUIT"
8,"JAM,MAGGI,BREAD,TEA"
9,"BREAD,MILK"


In [8]:
data.index.rename('TID', inplace=True)

In [9]:
trans_df = data.Item.str.split(",")
trans_df

TID
0                  [MILK, BREAD, BISCUIT]
1      [BREAD, MILK, BISCUIT, CORNFLAKES]
2                 [BREAD, TEA, BOURNVITA]
3               [JAM, MAGGI, BREAD, MILK]
4                   [MAGGI, TEA, BISCUIT]
5                 [BREAD, TEA, BOURNVITA]
6                [MAGGI, TEA, CORNFLAKES]
7            [MAGGI, BREAD, TEA, BISCUIT]
8                [JAM, MAGGI, BREAD, TEA]
9                           [BREAD, MILK]
10    [COFFEE, COCK, BISCUIT, CORNFLAKES]
11    [COFFEE, COCK, BISCUIT, CORNFLAKES]
12             [COFFEE, SUGER, BOURNVITA]
13                  [BREAD, COFFEE, COCK]
14                [BREAD, SUGER, BISCUIT]
15            [COFFEE, SUGER, CORNFLAKES]
16              [BREAD, SUGER, BOURNVITA]
17                 [BREAD, COFFEE, SUGER]
18                 [BREAD, COFFEE, SUGER]
19        [TEA, MILK, COFFEE, CORNFLAKES]
Name: Item, dtype: object

In [10]:
def prune(data,supp):
    
    df = data[data.supp_count >= supp] 
    return df

In [11]:
def count_itemset(transaction_df, itemsets):
    count_item = {}
    for item_set in itemsets:
        set_A = set(item_set)
        for row in trans_df:
            set_B = set(row)
            if set_B.intersection(set_A) == set_A: 
                if item_set in count_item.keys():
                    count_item[item_set] += 1   
                else:
                    count_item[item_set] = 1   
    data = pd.DataFrame()
    data['item_sets'] = count_item.keys()
    data['supp_count'] = count_item.values()
    return data

In [12]:
def count_item(trans_items): 
    count_ind_item = {}
    for row in trans_items:
        for i in range(len(row)):
            if row[i] in count_ind_item.keys():
                count_ind_item[row[i]] += 1
            else:
                count_ind_item[row[i]] = 1
                data = pd.DataFrame()
    data['item_sets'] = count_ind_item.keys()
    data['supp_count'] = count_ind_item.values()
    data = data.sort_values('item_sets')
    return data

In [13]:
def join(list_of_items):
    itemsets = []
    i = 1
    for entry in list_of_items:
        proceding_items = list_of_items[i:]
        for item in proceding_items:
            if(type(item) is str):
                if entry != item:
                    tuples = (entry, item)
                    itemsets.append(tuples)
            else:
                if entry[0:-1] == item[0:-1]:
                    tuples = entry+item[1:]
                    itemsets.append(tuples)
        i = i+1
    if(len(itemsets) == 0):
        return None
    return itemsets

In [14]:
def apriori(trans_data,supp=3, con=0.5):
    freq = pd.DataFrame()
    df = count_item(trans_data)
    while(len(df) != 0):
        df = prune(df, supp)
        if len(df) > 1 or (len(df) == 1 and int(df.supp_count >= supp)):
            freq = df
        itemsets = join(df.item_sets)
        if(itemsets is None):
            return freq
        df = count_itemset(trans_data, itemsets)
    return df

In [15]:
freq_item_sets = apriori(trans_df,4)

In [16]:
def calculate_conf(value1, value2):
    return round(int(value1)/int(value2) * 100, 2)

In [17]:
def strong_rules(freq_item_sets, threshold):

    confidences = {}
    for row in freq_item_sets.item_sets:
        for i in range(len(row)):
            for j in range(len(row)):
                 if i != j:
                    tuples = (row[i], row[j])
                    conf = calculate_conf(freq_item_sets[freq_item_sets.item_sets == row].supp_count, count_item(trans_df)[count_item(trans_df).item_sets == row[i]].supp_count)
                    confidences[tuples] = conf
                    conf_df = pd.DataFrame()
    conf_df['item_set'] = confidences.keys()
    conf_df['confidence'] = confidences.values()

    return conf_df[conf_df.confidence >= threshold]

In [18]:
strong_rules(freq_item_sets, 50.0)

,item_set,confidence
0,"(BISCUIT, BREAD)",57.14
3,"(MILK, BREAD)",80.00
5,"(SUGER, BREAD)",66.67
7,"(TEA, BREAD)",57.14
8,"(COFFEE, CORNFLAKES)",50.00
9,"(CORNFLAKES, COFFEE)",66.67
10,"(COFFEE, SUGER)",50.00
11,"(SUGER, COFFEE)",66.67
12,"(MAGGI, TEA)",80.00
13,"(TEA, MAGGI)",57.14


In [19]:
from functools import reduce
import operator

In [20]:
def interesting_rules(freq_item_sets):
    lifts = {}
    prob_of_items = []
    for row in freq_item_sets.item_sets:
        num_of_items = len(row)
        prob_all = freq_item_sets[freq_item_sets.item_sets == row].supp_count / 18
        for i in range(num_of_items):
            prob_of_items.append(count_item(trans_df)[count_item(trans_df).item_sets == row[i]].supp_count / 18)

In [21]:
def strong_rules(freq_item_sets, threshold):
    confidences = {}
    for row in freq_item_sets.item_sets:
        for i in range(len(row)):
            for j in range(len(row)):
                 if i != j:
                    tuples = (row[i], row[j])
                    conf = calculate_conf(freq_item_sets[freq_item_sets.item_sets == row].supp_count, count_item(trans_df)[count_item(trans_df).item_sets == row[i]].supp_count)
                    confidences[tuples] = conf
    conf_df = pd.DataFrame()
    conf_df['item_set'] = confidences.keys()
    conf_df['confidence'] = confidences.values()
    return conf_df[conf_df.confidence >= threshold]

In [22]:
strong_rules(freq_item_sets, 50.0)

,item_set,confidence
0,"(BISCUIT, BREAD)",57.14
3,"(MILK, BREAD)",80.00
5,"(SUGER, BREAD)",66.67
7,"(TEA, BREAD)",57.14
8,"(COFFEE, CORNFLAKES)",50.00
9,"(CORNFLAKES, COFFEE)",66.67
10,"(COFFEE, SUGER)",50.00
11,"(SUGER, COFFEE)",66.67
12,"(MAGGI, TEA)",80.00
13,"(TEA, MAGGI)",57.14


In [23]:
trans_df

TID
0                  [MILK, BREAD, BISCUIT]
1      [BREAD, MILK, BISCUIT, CORNFLAKES]
2                 [BREAD, TEA, BOURNVITA]
3               [JAM, MAGGI, BREAD, MILK]
4                   [MAGGI, TEA, BISCUIT]
5                 [BREAD, TEA, BOURNVITA]
6                [MAGGI, TEA, CORNFLAKES]
7            [MAGGI, BREAD, TEA, BISCUIT]
8                [JAM, MAGGI, BREAD, TEA]
9                           [BREAD, MILK]
10    [COFFEE, COCK, BISCUIT, CORNFLAKES]
11    [COFFEE, COCK, BISCUIT, CORNFLAKES]
12             [COFFEE, SUGER, BOURNVITA]
13                  [BREAD, COFFEE, COCK]
14                [BREAD, SUGER, BISCUIT]
15            [COFFEE, SUGER, CORNFLAKES]
16              [BREAD, SUGER, BOURNVITA]
17                 [BREAD, COFFEE, SUGER]
18                 [BREAD, COFFEE, SUGER]
19        [TEA, MILK, COFFEE, CORNFLAKES]
Name: Item, dtype: object

In [24]:
lst = []
for i in trans_df:
    lst.extend(i)

In [25]:
n = len(lst)
n

66

In [26]:
mp = {}
def countfreq(arr):
    for i in arr:
        if i in mp.keys():
            mp[i] = mp[i] + 1
        else:
            mp[i] = 1
countfreq(lst)

In [27]:
n = len(mp)
for i in mp:
    print(i," = ",mp[i])

MILK  =  5
BREAD  =  13
BISCUIT  =  7
CORNFLAKES  =  6
TEA  =  7
BOURNVITA  =  4
JAM  =  2
MAGGI  =  5
COFFEE  =  8
COCK  =  3
SUGER  =  6
